In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
import warnings

# Load environment variables
_ = load_dotenv(find_dotenv())

# Append the submodules path to the local libs directory
repo_dir = Path().resolve()
sys.path.append(str(repo_dir / 'libs'))

# Ensure the symlink exists (assuming setup_symlink.py has been executed)
symlink_path = repo_dir / 'libs' / 'NLP_on_multilingual_coin_datasets'
if not symlink_path.exists():
    print(f"Error: Symlink {symlink_path} does not exist. Run setup_symlink.py first.")
    sys.exit(1)

# Import the custom modules after ensuring symlink is in place
from NLP_on_multilingual_coin_datasets.cnt.io import Database_Connection
from modules.loading_preprocessed_designs import PreprocessingConfig

# Set up pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

# Suppress warnings
warnings.filterwarnings('ignore')

prep_cfg = PreprocessingConfig()


/Users/jf/miniconda3/envs/data_challenge_llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
database = prep_cfg.database

connection_string = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{database}"
engine = create_engine(connection_string)
dc = Database_Connection(connection_string)

In [3]:
# RE_query = """select design_id, 
# (select design_en from nlp_training_designs as nlp where re.design_id=nlp.id) as design_en_original,
# (select design_en_changed from nlp_training_designs as nlp where re.design_id=nlp.id) as design_en,
# (select name_en from nlp_list_entities as ner where ner.id=re.subject) as s, 
# (select class from nlp_list_entities as ner where ner.id=re.subject) as subject_class, 
# (select name_en from nlp_list_entities as ner where ner.id=re.predicate) as p, 
# (select name_en from nlp_list_entities as ner where ner.id=re.object) as o, 
# (select class from nlp_list_entities as ner where ner.id=re.object) as object_class
# from nlp_relation_extraction_en_v2 as re;
# """

RE_query = """select design_id, 
(select design_en_changed from nlp_training_designs as nlp where re.design_id=nlp.id) as design_en,
(select name_en from nlp_list_entities as ner where ner.id=re.subject) as s, 
(select class from nlp_list_entities as ner where ner.id=re.subject) as subject_class, 
(select name_en from nlp_list_entities as ner where ner.id=re.predicate) as p, 
(select name_en from nlp_list_entities as ner where ner.id=re.object) as o, 
(select class from nlp_list_entities as ner where ner.id=re.object) as object_class
from nlp_relation_extraction_en_v2 as re;
"""

df_RE_groundtruth= dc.create_own_query(RE_query)
# sort by design id ascending
df_RE_groundtruth = df_RE_groundtruth.sort_values(by=['design_id'])


df_RE_groundtruth.head()



,design_id,design_en,s,subject_class,p,o,object_class
0,9,Amphora with ribbed surface and crooked handles containing two ears of corn and poppy.,amphora,OBJECT,holding,poppy,PLANT
713,10,"Bust of youthful Anchialos, right, wearing taenia. Border of dots.",Anchialos,PERSON,wearing,taenia,OBJECT
715,24,"Bare-headed bust of Antoninus Pius, right, wearing cuirass and paludamentum.",Antoninus Pius,PERSON,wearing,paludamentum,OBJECT
714,24,"Bare-headed bust of Antoninus Pius, right, wearing cuirass and paludamentum.",Antoninus Pius,PERSON,wearing,cuirass,OBJECT
717,27,"Laureate bust of Antoninus Pius, right, wearing cuirass and paludamentum.",Antoninus Pius,PERSON,wearing,paludamentum,OBJECT


FOR ENGINE

In [4]:
df_RE_groundtruth.to_json(prep_cfg.json_path / "RE_groundtruth.json", orient="records")

df_spo_triples = pd.read_json(prep_cfg.json_path / "subject_predicate_object_triples.json", orient="records")

In [5]:
# filter df_spo for columns
columns = ['design_id', 's_o_id', 's', 'subject_class', 'predicate', 'o', 'object_class', 
           "validity_pred", "comment_pred", "implicit_pred", "design_en"
           ]

df_triples = df_spo_triples[columns].copy()
df_triples = df_triples.rename(columns={"predicate": "p"})

df_triples.head(5)

,design_id,s_o_id,s,subject_class,p,o,object_class,validity_pred,comment_pred,implicit_pred,design_en
0,1526,a,Apollo,PERSON,wearing,garment,OBJECT,1,Correct and meaningful SPO triple.,NULL,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left."
1,1526,b,Apollo,PERSON,holding,lyre,OBJECT,1,Correct and meaningful SPO triple.,NULL,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left."
2,1526,c,Apollo,PERSON,on,column,OBJECT,-1,Predicate 'on' is not plausible in this context.,standing_on,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left."
3,1527,a,club,OBJECT,over,lion skin,OBJECT,1,Correct and meaningful SPO triple.,NULL,"Upright club, lion skin to left over it."
4,1528,a,Apollo,PERSON,wearing,chlamys,OBJECT,1,Correct and meaningful SPO triple.,NULL,"Wreath bust of Apollo, left, wearing chlamys; in front, bow. Border of dots."


In [6]:
df_triples.to_json(prep_cfg.json_path / "RE_new_datachallenge.json", orient="records")

df_RE = pd.merge(df_triples, df_RE_groundtruth, on="design_id", how="inner")
df_RE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6232 entries, 0 to 6231
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   design_id        6232 non-null   int64 
 1   s_o_id           6232 non-null   object
 2   s_x              6232 non-null   object
 3   subject_class_x  6232 non-null   object
 4   p_x              6232 non-null   object
 5   o_x              6232 non-null   object
 6   object_class_x   6232 non-null   object
 7   validity_pred    6232 non-null   int64 
 8   comment_pred     6232 non-null   object
 9   implicit_pred    6232 non-null   object
 10  design_en_x      6232 non-null   object
 11  design_en_y      6232 non-null   object
 12  s_y              6232 non-null   object
 13  subject_class_y  6232 non-null   object
 14  p_y              6232 non-null   object
 15  o_y              6232 non-null   object
 16  object_class_y   6232 non-null   object
dtypes: int64(2), object(15)
memory us

In [7]:
df_RE.head(5)

,design_id,s_o_id,s_x,subject_class_x,p_x,o_x,object_class_x,validity_pred,comment_pred,implicit_pred,design_en_x,design_en_y,s_y,subject_class_y,p_y,o_y,object_class_y
0,1526,a,Apollo,PERSON,wearing,garment,OBJECT,1,Correct and meaningful SPO triple.,NULL,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.","Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",Apollo,PERSON,holding,lyre,OBJECT
1,1526,a,Apollo,PERSON,wearing,garment,OBJECT,1,Correct and meaningful SPO triple.,NULL,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.","Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",Apollo,PERSON,wearing,garment,OBJECT
2,1526,b,Apollo,PERSON,holding,lyre,OBJECT,1,Correct and meaningful SPO triple.,NULL,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.","Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",Apollo,PERSON,holding,lyre,OBJECT
3,1526,b,Apollo,PERSON,holding,lyre,OBJECT,1,Correct and meaningful SPO triple.,NULL,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.","Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",Apollo,PERSON,wearing,garment,OBJECT
4,1526,c,Apollo,PERSON,on,column,OBJECT,-1,Predicate 'on' is not plausible in this context.,standing_on,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.","Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",Apollo,PERSON,holding,lyre,OBJECT


In [8]:
df_RE_compare = df_RE.rename(columns={
    's_x': 's_new',
    'subject_class_x': 's_class_new',
    'p_x': 'p_new',
    'o_x': 'o_new',
    'object_class_x': 'o_class_new',
    's_y': 's_old',
    'subject_class_y': 's_class_old',
    'p_y': 'p_old',
    'o_y': 'o_old',
    'object_class_y': 'o_class_old',
    'design_en_x': 'design_en'
})[[
    'design_id', 's_o_id', 's_new', 's_class_new', 'p_new', 'o_new', 'o_class_new',
    's_old', 's_class_old', 'p_old', 'o_old', 'o_class_old',
    'design_en', 'validity_pred', 'comment_pred', 'implicit_pred'
]].copy()

In [9]:
df_RE_compare.to_json(prep_cfg.json_path / "RE_compare_groundtruth_vs_datachallenge.json", orient="records")
df_RE_compare.head(5)

,design_id,s_o_id,s_new,s_class_new,p_new,o_new,o_class_new,s_old,s_class_old,p_old,o_old,o_class_old,design_en,validity_pred,comment_pred,implicit_pred
0,1526,a,Apollo,PERSON,wearing,garment,OBJECT,Apollo,PERSON,holding,lyre,OBJECT,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",1,Correct and meaningful SPO triple.,NULL
1,1526,a,Apollo,PERSON,wearing,garment,OBJECT,Apollo,PERSON,wearing,garment,OBJECT,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",1,Correct and meaningful SPO triple.,NULL
2,1526,b,Apollo,PERSON,holding,lyre,OBJECT,Apollo,PERSON,holding,lyre,OBJECT,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",1,Correct and meaningful SPO triple.,NULL
3,1526,b,Apollo,PERSON,holding,lyre,OBJECT,Apollo,PERSON,wearing,garment,OBJECT,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",1,Correct and meaningful SPO triple.,NULL
4,1526,c,Apollo,PERSON,on,column,OBJECT,Apollo,PERSON,holding,lyre,OBJECT,"Apollo standing facing, head left,wearing long garment, holding lyre with left hand on a short column to his left.",-1,Predicate 'on' is not plausible in this context.,standing_on


# Comparison new triples vs Ground truth


- create json file in (exisiting) RE formatting
    - triples, keys and some values?
    - map verbs/predicates to classes

- Compare (some) triples with RE ground truth
    - Read RE ground truth
    - Check if some design_ids match
        - if yes compare them
        - if not, compute some
    - create one Df merge over all matching triples.



In [10]:
df_spo_triples = pd.read_json(prep_cfg.json_path / "subject_predicate_object_triples.json", orient="records")
df_spo_triples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   design_id            8623 non-null   int64  
 1   s_o_id               8623 non-null   object 
 2   s                    8623 non-null   object 
 3   subject_class        8623 non-null   object 
 4   predicate            8623 non-null   object 
 5   o                    8623 non-null   object 
 6   object_class         8623 non-null   object 
 7   validity_pred        8623 non-null   int64  
 8   comment_pred         8623 non-null   object 
 9   implicit_pred        8623 non-null   object 
 10  validity_sop         8623 non-null   int64  
 11  comment_sop          8623 non-null   object 
 12  design_en            8623 non-null   object 
 13  new_list_of_strings  8623 non-null   object 
 14  relevance            8623 non-null   int64  
 15  correctness          8623 non-null   i

In [11]:
# filter all predicate == "NULL" 

df_spo_triples = df_spo_triples[df_spo_triples["predicate"] != "NULL"]

df_spo_triples["validity_pred"].value_counts()

validity_pred
 1    5936
 0     716
-1     411
Name: count, dtype: int64

In [12]:
one = 5845
zero = 706
minus = 390
overall = one + zero + minus

rel_one = round(one / overall, 2)
rel_zero = round(zero / overall, 2)
rel_minus = round(minus / overall, 2)

print(f"relative 1: {rel_one}, \nrelative 0: {rel_zero}, \nrelative -1: {rel_minus}")

relative 1: 0.84, 
relative 0: 0.1, 
relative -1: 0.06


In [13]:
columns_new = ['design_id', 's', 'subject_class', 'predicate', 'o', 'object_class', 'validity_pred', 'implicit_pred', 'design_en']
df_triples_new = df_spo_triples[columns_new].copy()
df_triples_new = df_triples_new.rename(columns={"predicate": "p"})
df_triples_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7063 entries, 0 to 8609
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   design_id      7063 non-null   int64 
 1   s              7063 non-null   object
 2   subject_class  7063 non-null   object
 3   p              7063 non-null   object
 4   o              7063 non-null   object
 5   object_class   7063 non-null   object
 6   validity_pred  7063 non-null   int64 
 7   implicit_pred  7063 non-null   object
 8   design_en      7063 non-null   object
dtypes: int64(2), object(7)
memory usage: 551.8+ KB


In [14]:
# Aggregation functions
def agg_long(x):
    return list(zip(x['s'], x['subject_class'], x['p'], x['o'], x['object_class'], x['validity_pred'], x['implicit_pred']))

def agg_short(x):
    return list(zip(x['s'], x['subject_class'], x['p'], x['o'], x['object_class']))

# Group by 'design_id' and aggregate
df_aggregated = df_triples_new.groupby(['design_id', 'design_en']).apply(lambda x: pd.Series({
    'l_spo_long': agg_long(x),
    'l_spo_short': agg_short(x)
})).reset_index()

df_aggregated.head(5)


,design_id,design_en,l_spo_long,l_spo_short
0,8,Prize amphora on ornamental stand; within linear square and incuse square.,"[(amphora, OBJECT, on, stand, OBJECT, 1, NULL)]","[(amphora, OBJECT, on, stand, OBJECT)]"
1,9,Amphora with ribbed surface and crooked handleholding two corn and poppy.,"[(Amphora, OBJECT, holding, corn, PLANT, -1, decorated_with), (Amphora, OBJECT, holding, poppy, PLANT, -1, decorated_with)]","[(Amphora, OBJECT, holding, corn, PLANT), (Amphora, OBJECT, holding, poppy, PLANT)]"
2,10,"Bust of youthful Anchialos, right, wearing taenia. Border of dots.","[(Anchialos, PERSON, wearing, taenia, OBJECT, 1, NULL)]","[(Anchialos, PERSON, wearing, taenia, OBJECT)]"
3,11,"Inverted anchor; under left fluke, crayfish, under right fluke, ethnicon.","[(anchor, OBJECT, under, crayfish, ANIMAL, 1, NULL), (anchor, OBJECT, under, ethnicon, OBJECT, 1, NULL)]","[(anchor, OBJECT, under, crayfish, ANIMAL), (anchor, OBJECT, under, ethnicon, OBJECT)]"
4,12,"Inverted anchor; under left fluke, crayfish, under right fluke, ethnicon; all within circular incuse.","[(anchor, OBJECT, under, crayfish, ANIMAL, 1, NULL), (anchor, OBJECT, under, ethnicon, OBJECT, 1, NULL)]","[(anchor, OBJECT, under, crayfish, ANIMAL), (anchor, OBJECT, under, ethnicon, OBJECT)]"


In [16]:
# load groundtruth data from json   
df_RE_groundtruth = pd.read_json(prep_cfg.json_path / "RE_groundtruth.json", orient="records")
df_RE_groundtruth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2357 entries, 0 to 2356
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   design_id      2357 non-null   int64 
 1   design_en      2357 non-null   object
 2   s              2357 non-null   object
 3   subject_class  2357 non-null   object
 4   p              2357 non-null   object
 5   o              2357 non-null   object
 6   object_class   2357 non-null   object
dtypes: int64(1), object(6)
memory usage: 129.0+ KB


In [17]:
# Group by 'design_id' and 'design_en' and aggregate
df_aggregated_groundtruth = df_RE_groundtruth.groupby(['design_id', 'design_en']).apply(lambda x: pd.Series({
    'l_spo_short': agg_short(x)
})).reset_index()

print(df_aggregated_groundtruth.info())
df_aggregated_groundtruth.drop(columns="design_en", inplace=True)
df_aggregated_groundtruth.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   design_id    1297 non-null   int64 
 1   design_en    1297 non-null   object
 2   l_spo_short  1297 non-null   object
dtypes: int64(1), object(2)
memory usage: 30.5+ KB
None


,design_id,l_spo_short
0,9,"[(amphora, OBJECT, holding, poppy, PLANT)]"
1,10,"[(Anchialos, PERSON, wearing, taenia, OBJECT)]"
2,24,"[(Antoninus Pius, PERSON, wearing, paludamentum, OBJECT), (Antoninus Pius, PERSON, wearing, cuirass, OBJECT)]"
3,27,"[(Antoninus Pius, PERSON, wearing, paludamentum, OBJECT), (Antoninus Pius, PERSON, wearing, cuirass, OBJECT)]"
4,28,"[(Antoninus Pius, PERSON, wearing, paludamentum, OBJECT)]"
5,33,"[(Aphrodite, PERSON, holding, apple, PLANT)]"
6,36,"[(Eros, PERSON, seated_on, dolphin, ANIMAL)]"
7,38,"[(Apollo, PERSON, holding, arrow, OBJECT), (Apollo, PERSON, holding, bow, OBJECT)]"
8,39,"[(Apollo, PERSON, wearing, chlamys, OBJECT), (Apollo, PERSON, holding, arrow, OBJECT)]"
9,61,"[(Apollo, PERSON, seated_on, knee, OBJECT), (Apollo, PERSON, resting_on, rock, OBJECT), (Apollo, PERSON, holding, lyre, OBJECT)]"


In [18]:
# Merge DataFrames on 'design_id'
df_merged = pd.merge(df_aggregated, df_aggregated_groundtruth, on='design_id', suffixes=('_new', '_gt'))
print(df_merged.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   design_id        752 non-null    int64 
 1   design_en        752 non-null    object
 2   l_spo_long       752 non-null    object
 3   l_spo_short_new  752 non-null    object
 4   l_spo_short_gt   752 non-null    object
dtypes: int64(1), object(4)
memory usage: 29.5+ KB
None


In [19]:
# Function to compare lists
def compare_lists(row):
    list_agg = row['l_spo_short_new']
    list_gt = row['l_spo_short_gt']
    
    set_agg = set(list_agg)
    set_gt = set(list_gt)
    
    if set_agg == set_gt:
        return 0
    
    agg_dict = {(s, o): p for s, _, p, o, _ in list_agg}
    gt_dict = {(s, o): p for s, _, p, o, _ in list_gt}
    
    if len(list_agg) == len(list_gt) and all((s, o) in agg_dict and agg_dict[(s, o)] != p for (s, o), p in gt_dict.items()):
        return 0.1
    
    if len(list_agg) > len(list_gt) and all((s, o) in agg_dict and agg_dict[(s, o)] != p for (s, o), p in gt_dict.items()):
        return 1.1
    
    if set_agg.issuperset(set_gt):
        return 1
    
    return -1

0.0: Exact match of l_spo_short lists in both DataFrames.
1.0: Aggregated list contains all elements of the ground truth list plus more.
-1.0: Aggregated list does not cover all elements of the ground truth list.
0.1: Lists are the same length; s and o match but p values differ.
1.1: Aggregated list is longer; s and o match but p values differ.

In [20]:
df_merged['comparison_result'] = df_merged.apply(compare_lists, axis=1)
print(df_merged['comparison_result'].value_counts())
print(df_merged.info())  


comparison_result
-1.0    303
 1.0    245
 0.0    177
 1.1     17
 0.1     10
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   design_id          752 non-null    int64  
 1   design_en          752 non-null    object 
 2   l_spo_long         752 non-null    object 
 3   l_spo_short_new    752 non-null    object 
 4   l_spo_short_gt     752 non-null    object 
 5   comparison_result  752 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 35.4+ KB
None


In [21]:
one = 262
zero = 186
minus = 301
overall = one + zero + minus

rel_one = round(one / overall, 2)
rel_zero = round(zero / overall, 2)
rel_minus = round(minus / overall, 2)

print(f"rel match ++: {rel_one}, \nrel match: {rel_zero}, \nrel diff: {rel_minus}")

rel match ++: 0.35, 
rel match: 0.25, 
rel diff: 0.4


In [23]:
#save df_merged to json
df_merged.to_json(prep_cfg.json_path / "RE_compare_groundtruth_vs_datachallenge_aggregated.json", orient="records")

In [24]:
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   design_id          752 non-null    int64  
 1   design_en          752 non-null    object 
 2   l_spo_long         752 non-null    object 
 3   l_spo_short_new    752 non-null    object 
 4   l_spo_short_gt     752 non-null    object 
 5   comparison_result  752 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 35.4+ KB


In [25]:
df_merged

,design_id,design_en,l_spo_long,l_spo_short_new,l_spo_short_gt,comparison_result
0,9,Amphora with ribbed surface and crooked handleholding two corn and poppy.,"[(Amphora, OBJECT, holding, corn, PLANT, -1, decorated_with), (Amphora, OBJECT, holding, poppy, PLANT, -1, decorated_with)]","[(Amphora, OBJECT, holding, corn, PLANT), (Amphora, OBJECT, holding, poppy, PLANT)]","[(amphora, OBJECT, holding, poppy, PLANT)]",-1.0
1,10,"Bust of youthful Anchialos, right, wearing taenia. Border of dots.","[(Anchialos, PERSON, wearing, taenia, OBJECT, 1, NULL)]","[(Anchialos, PERSON, wearing, taenia, OBJECT)]","[(Anchialos, PERSON, wearing, taenia, OBJECT)]",0.0
2,27,"Wreath bust of Antoninus Pius, right, wearing cuirass and paludamentum.","[(Antoninus Pius, PERSON, wearing, cuirass, OBJECT, 1, NULL), (Antoninus Pius, PERSON, wearing, paludamentum, OBJECT, 1, NULL)]","[(Antoninus Pius, PERSON, wearing, cuirass, OBJECT), (Antoninus Pius, PERSON, wearing, paludamentum, OBJECT)]","[(Antoninus Pius, PERSON, wearing, paludamentum, OBJECT), (Antoninus Pius, PERSON, wearing, cuirass, OBJECT)]",0.0
3,28,"Wreath bust of Antoninus Pius, right, wearing paludamentum.","[(Antoninus Pius, PERSON, wearing, paludamentum, OBJECT, 1, NULL)]","[(Antoninus Pius, PERSON, wearing, paludamentum, OBJECT)]","[(Antoninus Pius, PERSON, wearing, paludamentum, OBJECT)]",0.0
4,33,"Half-nude Aphrodite standing facing, head left, holding apple in raised right hand.","[(Aphrodite, PERSON, holding, apple, PLANT, 1, NULL)]","[(Aphrodite, PERSON, holding, apple, PLANT)]","[(Aphrodite, PERSON, holding, apple, PLANT)]",0.0
5,36,"Nude Aphrodite standing facing, head right, holding her breast with right hand and pudenda with left hand; to left, Eros, seated_on a dolphin downwards.","[(Aphrodite, PERSON, holding, breast, OBJECT, 1, NULL), (Aphrodite, PERSON, holding, pudenda, OBJECT, 1, NULL), (Eros, PERSON, seated_on, dolphin, ANIMAL, 1, NULL)]","[(Aphrodite, PERSON, holding, breast, OBJECT), (Aphrodite, PERSON, holding, pudenda, OBJECT), (Eros, PERSON, seated_on, dolphin, ANIMAL)]","[(Eros, PERSON, seated_on, dolphin, ANIMAL)]",1.0
6,38,"Nude Apollo advancing right, holding arrow andholding bow in his left hand.","[(Apollo, PERSON, holding, arrow, OBJECT, 1, NULL), (Apollo, PERSON, holding, bow, OBJECT, 1, NULL)]","[(Apollo, PERSON, holding, arrow, OBJECT), (Apollo, PERSON, holding, bow, OBJECT)]","[(Apollo, PERSON, holding, arrow, OBJECT), (Apollo, PERSON, holding, bow, OBJECT)]",0.0
7,39,"Nude Apollo advancing right, wearing fluttering chlamys, drawing arrow in right hand from bow in left hand.","[(Apollo, PERSON, wearing, chlamys, OBJECT, 1, NULL), (Apollo, PERSON, drawing, arrow, OBJECT, 1, NULL), (Apollo, PERSON, drawing, bow, OBJECT, 0, NULL)]","[(Apollo, PERSON, wearing, chlamys, OBJECT), (Apollo, PERSON, drawing, arrow, OBJECT), (Apollo, PERSON, drawing, bow, OBJECT)]","[(Apollo, PERSON, wearing, chlamys, OBJECT), (Apollo, PERSON, holding, arrow, OBJECT)]",-1.0
8,61,"Nude Apollo seated_on right on rock, playing lyre set on his left knee. Ground line.","[(Apollo, PERSON, seated_on, rock, OBJECT, 1, NULL), (Apollo, PERSON, playing, lyre, OBJECT, 1, NULL)]","[(Apollo, PERSON, seated_on, rock, OBJECT), (Apollo, PERSON, playing, lyre, OBJECT)]","[(Apollo, PERSON, seated_on, knee, OBJECT), (Apollo, PERSON, resting_on, rock, OBJECT), (Apollo, PERSON, holding, lyre, OBJECT)]",-1.0
9,65,"Nude Apollo standing left, holding patera in outstretched right hand and bow in left hand.","[(Apollo, PERSON, holding, patera, OBJECT, 1, NULL), (Apollo, PERSON, holding, bow, OBJECT, 1, NULL)]","[(Apollo, PERSON, holding, patera, OBJECT), (Apollo, PERSON, holding, bow, OBJECT)]","[(Apollo, PERSON, holding, patera, OBJECT), (Apollo, PERSON, holding, bow, OBJECT)]",0.0
